### Generators
NOTE: In python you will always find this pattern: <b>top-level function or syntax and a corresponding `__method__` function</b><br>
```x()         __call__ protocol```

### Example: Add 

In [1]:
def add1(x, y): # as a function
    return x + y

class Adder: # as a class
    def __call__(self, x, y):
        return x + y

add2 = Adder()

In [2]:
add1(1, 2)

3

In [3]:
add2(1, 2)

3

#### What is the difference between add1 and add2? 
Functionally, they're the same. But one is a function and one is a class.
<br><br>
If you wanted to add some statement behaviour:

In [4]:
class Adder:
    def __init__(self):
        self.z = 0

    def __call__(self, x, y):
        self.z += 1
        return x + y + self.z

i.e. there is some nice syntax and then some object model that everything sits in.

## An example that has long computation time
A function that takes a lot of time to do something i.e. network request
```
def load_data():
    rows = []
    while db.read():
        rows.append( ... )
```

In [2]:
from time import sleep # let's mimick reading a database/some heavy complex process
def compute():
    rv = []
    for i in range(10):
        sleep(.5)
        rv.append(i)
    return rv

In [3]:
compute()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

<b>Eagerness vs Laziness</b>

When you run this, it will take 5 seconds to run.
- time: if we only want the first value, it will still take 5 seconds to run.
- memory: it will require 10 units of memory (10 integers in a list), even if we only care about the first element.

This is the notion of eagerness - irrespective of what you care about in the computation, it will always take the full amount of memory and time. It eagerly gives you the entire result and you're waiting for the entire result before you can process anything.
<br><br>
This is undesireable:
- If you have 1 million entries, you need to wait the entire time even if you want to start processing elements one by one.
- You have to hold gigabytes of memory whilst you process one by one.

In [4]:
# as a class

class Compute:
    def __class__(self):
        rv = []
        for i in range(10):
            sleep(.5)
            rv.append(i)
        return rv

compute = Compute()

#### So if we only want one element at a time... where have we seen this?
For Loops


A basic loop --> get one element at a time i.e. `for i in mylist`

Remember: <b>top-level syntax or function --> underscore method</b><br>
```
for x in xs:
    pass
```
The above looks like this under the covers:
```
x1 = iter(xs)           --> __iter__
while True:
    x = next(x1)        --> __next__
```
So we can take a class and add an iter and a next. and suddenly it can
    be iterated over.


In [6]:
class Compute:
    def __iter__(self):
        self.last = 0
        return self
    
    def __next__(self):
        rv = self.last
        self.last += 1
        
        if self.last > 10:
            raise StopIteration()
            
        sleep(.5)
        return rv

In [7]:
# Now we are only returning one element in the list each time
for val in Compute():
    print(val)

0
1
2
3
4
5
6
7
8
9


In [9]:
# So lets rewrite the original compute function early above.
def compute(): # NOTE: compute can be iterated over
    for i in range(10):
        sleep(.5)
        yield(i)

### Summary
Taken the original function formulation and modified to a <b>generated</b> formulation.
<br><br>
i.e. some library code runs, then some user code, then some library code and so on...
<br><br>
This is the main idea of co-routines and the idea of generators.<br>
i.e. iterleaving of library and user code --> generator yields and returns value and waits for the user code and then generator yields another...

There is one more important feature of a generator...
## Example 2: APIs

In [11]:
# Often see API's that look like this

class Api:
    def run_this_first(self):
        first()
    def run_this_second(self):
        second()
    def run_this_last(self):
        last()

And the documentation says make sure to run it first. then second. then last.
otherwise it will all break.
<br><br>
But nothing stops you from doing this in the wrong order i.e.:
```
Api.run_this_last()
Api.run_this_first()
```
In this case, we'd want some interleaving.<br>
NOTE: if we didn't want to have interleaving between the three steps in the API then we wouldn't have 3 separate functions. We would've just written it all together.

In [12]:
def api():
    first()
    yield # yield control (no value necessarily)
    second()
    yield
    last()


Generators are a mechanism by which you can create code that interleaves with other code and also enforces interleaving (co-routines). forces sequencing.